In [1]:
import os
import xlutils, xypath
import databaker
import databaker.constants
from databaker.constants import *
import databaker.databakersolo as ds
from databaker.jupybakeutils import savepreviewhtml, writetechnicalCSV, procrows, TechnicalCSV, yield_dimension_values 
from databaker.jupybakeutils import savepreviewhtmlBAGS, HDim
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup


In [2]:
import zipfile, io, re, os
from tempfile import NamedTemporaryFile

dirname = "tablep1zips"
dfiles = os.listdir(dirname)
dfiles.sort(reverse=True)
previewfile = "preview.html"

# each zip file has name and CV version
zipfilefile = [ ] # [ (zipfile, main, CV) ]
for dfile in dfiles:
    zfile = os.path.join(dirname, dfile)
    zdir = zipfile.ZipFile(zfile)
    zff = zdir.namelist()
    zff = [z  for z in zff  if z[-4:] == ".xls"]   # strip out docx versions
    zff.sort()
    assert len(zff) == 2, zff
    zmain, zcv = zff
    assert zcv[-7:] == " CV.xls", zff
    assert zcv[:-7].replace("a", "b") == zmain[:-4].replace("a", "b"), zff
    zipfilefile.append((zfile, zmain, zcv))
        
zipfilefile


[('tablep1zips/2015-2015provisionaltablep1.zip',
  'PROV - Pension Provision by Earnings & Age Group Table P1.1a   Pension Type 2015.xls',
  'PROV - Pension Provision by Earnings & Age Group Table P1.1b   Pension Type 2015 CV.xls'),
 ('tablep1zips/2014-2014revisedtablep1.zip',
  'REVISED - Pension Provision by Earnings & Age Group Table P1.1a   Pension Type 2014.xls',
  'REVISED - Pension Provision by Earnings & Age Group Table P1.1b   Pension Type 2014 CV.xls'),
 ('tablep1zips/2013revised-2013-revised-table-p1.zip',
  'REVISED - Pension Provision by Earnings & Age Group new Table P1.1a   Pension Type 2013.xls',
  'REVISED - Pension Provision by Earnings & Age Group new Table P1.1b   Pension Type 2013 CV.xls'),
 ('tablep1zips/2012revised-2012-revised-table-p1.zip',
  'REVISED - Pension Provision by Earnings & Age Group Table P1.1a   Pension Type 2012.xls',
  'REVISED - Pension Provision by Earnings & Age Group Table P1.1b   Pension Type 2012 CV.xls'),
 ('tablep1zips/2011revisedsoc2010-

In [3]:
from tempfile import NamedTemporaryFile

def tablesetfromzip(zfile, fname):
    zdir = zipfile.ZipFile(zfile)
    z = zdir.read(zmain)
    tf = NamedTemporaryFile("w+b", suffix=".xls")
    tf.write(z)
    return xypath.loader.table_set(tf.name, extension='xls')

# Now go through all and get the sheets for each of these (wastefully discarding after loading)
for zfile, zmain, zcv in zipfilefile:
    tablesetmain = tablesetfromzip(zfile, zmain)
    tablesetcv = tablesetfromzip(zfile, zcv)
    print(zfile, tablesetmain.names,tablesetcv.names)


tablep1zips/2015-2015provisionaltablep1.zip {'Female', 'Notes', 'All', 'Glossary', 'Male'} {'Female', 'Notes', 'All', 'Glossary', 'Male'}
tablep1zips/2014-2014revisedtablep1.zip {'Female', 'Notes', 'All', 'Glossary', 'Male'} {'Female', 'Notes', 'All', 'Glossary', 'Male'}
tablep1zips/2013revised-2013-revised-table-p1.zip {'Female', 'All', 'Male'} {'Female', 'All', 'Male'}
tablep1zips/2012revised-2012-revised-table-p1.zip {'Female', 'All', 'Male'} {'Female', 'All', 'Male'}
tablep1zips/2011revisedsoc2010-2011-revised-table-p1--soc-2010-.zip {'Female', 'All', 'Male'} {'Female', 'All', 'Male'}
tablep1zips/2011provisionalsoc2000-2011-provisional-table-p1.zip {'Female', 'All', 'Male'} {'Female', 'All', 'Male'}
tablep1zips/2010revised-2010-revised-table-p1.zip {'Female', 'All', 'Male'} {'Female', 'All', 'Male'}
tablep1zips/2009-2009-table-p1.zip {'Female', 'All', 'Male'} {'Female', 'All', 'Male'}
tablep1zips/2008-2008-table-p1.zip {'Female', 'All', 'Male'} {'Female', 'All', 'Male'}
tablep1zips

In [4]:
# Let's take one table from this list
zfile, zmain, zcv = zipfilefile[0]
tablesetmain = tablesetfromzip(zfile, zmain)
tabs = list(xypath.loader.get_sheets(tablesetmain, "All"))
tab = tabs[0]

In [5]:
previewfile = "preview.html"
savepreviewhtmlBAGS(tab, previewfile)

opening file preview.html
table 'All' written


In [6]:
anchor = tab.is_bold().filter(contains_string('Description')).assert_one()

In [7]:
cat1 = anchor.fill(DOWN)
cat2 = cat1.is_bold()
savepreviewhtmlBAGS([cat1, cat2], previewfile)

opening file preview.html
table 'All' written


In [8]:
dim1 = HDim(cat1, "cat1", DIRECTLY, LEFT)
dim2 = HDim(cat2, "cat2", CLOSEST, UP)

# apply the over-rides into the dimension !!!!!!!!
for c in dim1.hbagset:
    if c.is_bold():
        dim1.cellvalueoverride[c._cell] = "All"

dimensions = [ 
        dim2, dim1, 
                ]
obs = anchor.fill(RIGHT).fill(DOWN).is_number()

conversionsegment = (tab, dimensions, obs)
savepreviewhtml(conversionsegment, batchcelllookup, previewfile)
conversionsegments = [conversionsegment]

opening file preview.html
table 'All' written
javascript calculated


In [9]:
k = list(dim1.hbagset)[0]
k._cell

<A7 'All employees'>

In [10]:
outputfile = "output.csv"        
print("converting and %d conversion segments" % (len(conversionsegments)))
convertedrows = [ ]
for conversionsegment in conversionsegments:
    print("conversion segment size %d" % len(conversionsegment[2]))
    rows = procrows(conversionsegment, batchcelllookup)
    convertedrows.append(rows)
    
# Should detect errors and throw up preview with cells marked
# (though actually the errors manifest further upstream, normally)
    
# here we can do further sorting and postprocessing and allocating to different files 


converting and 1 conversion segments
conversion segment size 903


In [11]:
rows = convertedrows[0]
rows[::20]

[{'cat2': 'All employees', 'cat1': 'All', -9: 6510.0},
 {'cat2': 'All employees', 'cat1': '     Less than £100', -9: 69.0},
 {'cat2': 'All employees', 'cat1': '     £100 - £200', -9: 83.0},
 {'cat2': 'All employees', 'cat1': '     £200 - £300', -9: 1641.0},
 {'cat2': 'All employees', 'cat1': '     £400 - £500', -9: 171.0},
 {'cat2': 'All employees', 'cat1': '     £500 - £600', -9: 448.0},
 {'cat2': 'All employees', 'cat1': '     £600 and over', -9: 411.0},
 {'cat2': '16 - 21', 'cat1': '     Less than £100', -9: 8.0},
 {'cat2': '16 - 21', 'cat1': '     £200 - £300', -9: 3.5},
 {'cat2': '16 - 21', 'cat1': '     £400 - £500', -9: 66.0},
 {'cat2': '22 - 29', 'cat1': 'All', -9: 42.0},
 {'cat2': '22 - 29', 'cat1': '     £100 - £200', -9: 16.5},
 {'cat2': '22 - 29', 'cat1': '     £200 - £300', -9: 42.0},
 {'cat2': '22 - 29', 'cat1': '     £400 - £500', -9: 3.6},
 {'cat2': '22 - 29', 'cat1': '     £500 - £600', -9: 18.6},
 {'cat2': '22 - 29', 'cat1': '     £600 and over', -9: 0.9},
 {'cat2': '